In [1]:
import glob
import time
import numpy as np
np.set_printoptions(suppress=True)

from SmoothSparseWeights_CVXOPT import *

In [2]:
# exampleName="cheburashka"
# poses_folder="poses-2"
# poses_ind=7
# clip_k=4
# handle_num=11


# exampleName="cow"
# poses_folder="poses-2"
# poses_ind=5
# clip_k=4
# handle_num=15


exampleName="wolf"
poses_folder="poses-1"
poses_ind=2
clip_k=4
handle_num=21



weights = {'W_spatial': 0.0}

In [3]:
#### read obj file

### rest_poses:
vertices_obj_path="./models/"+exampleName+"-all/"+exampleName+".obj"
mesh0=TriMesh.FromOBJ_FileName(vertices_obj_path)

### other poses
vertices_obj_dir="./models/"+exampleName+"-all/"+poses_folder+"/"
filenames=glob.glob(vertices_obj_dir+"*.obj")
print(filenames)

mesh1_list=[]
for i in range(len(filenames)):
    mesh1_list.append(TriMesh.FromOBJ_FileName(filenames[i]))


vertices1=vertices1_temp=np.hstack((np.asarray(mesh0.vs),np.ones((len(mesh0.vs),1))))
vertices2=vertices2_temp=np.asarray(mesh1_list[0].vs)
for i in range(1,len(mesh1_list)):
    vertices1=np.hstack((vertices1, vertices1_temp))
    vertices2=np.hstack((vertices2, np.asarray(mesh1_list[i].vs)))
print(vertices1.shape) 
print(vertices2.shape) 


['./models/wolf-all/poses-1/wolf-1.obj', './models/wolf-all/poses-1/wolf-2.obj', './models/wolf-all/poses-1/wolf-3.obj', './models/wolf-all/poses-1/wolf-4.obj', './models/wolf-all/poses-1/wolf-5.obj', './models/wolf-all/poses-1/wolf-6.obj']
(5075, 24)
(5075, 18)


In [4]:
#### read our output of simplex_hull.py 
results_path="./results_pipeline/"+exampleName+"-"+str(handle_num)+"/result.txt"
endmembers, mixing_weights=load_result(results_path)
### endmembers shape is handle * poses * 12??

# ### this is to make sure the 3*4 or 4*3 error does not happen.
# endmembers=endmembers.reshape((-1, 4, 3))
# endmembers=np.transpose(endmembers, (0,2,1))
# endmembers=endmembers.reshape((len(mixing_weights), -1, 12))


endmembers=endmembers.reshape((len(mixing_weights),-1))
mixing_weights=mixing_weights.T


print (endmembers.shape)
print (mixing_weights.shape)
print (mixing_weights[:3])
mixing_weights_clip, grad_zero_indices=clip_first_k_values(mixing_weights, clip_k)
print (mixing_weights_clip[:3])

print (grad_zero_indices[:len(endmembers)*3])

print (mixing_weights_clip.ravel()[grad_zero_indices].sum())

(21, 72)
(5075, 21)
[[0.1589514  0.0000593  0.00474596 0.00048292 0.0000278  0.03343458
  0.07699286 0.17446554 0.03199314 0.00009963 0.01256247 0.12470764
  0.09002658 0.0000002  0.00000122 0.00001448 0.01316137 0.00123948
  0.00000147 0.00000064 0.27703134]
 [0.15856047 0.00005942 0.00472487 0.0004812  0.00002775 0.03355878
  0.07708042 0.17432981 0.03208895 0.00009982 0.01256983 0.12475939
  0.08994991 0.0000002  0.00000122 0.00001445 0.01320413 0.00123941
  0.00000147 0.00000064 0.27724787]
 [0.15566619 0.00006021 0.00463952 0.00047399 0.00002755 0.03405925
  0.07742135 0.17335098 0.03247172 0.00010021 0.01264691 0.12530878
  0.08969101 0.0000002  0.00000124 0.00001417 0.01345853 0.00124074
  0.00000144 0.00000064 0.27936539]]
[[0.1589514  0.         0.         0.         0.         0.
  0.         0.17446554 0.         0.         0.         0.12470764
  0.         0.         0.         0.         0.         0.
  0.         0.         0.27703134]
 [0.15856047 0.         0.         

In [5]:
scale=find_scale(vertices1)
print (scale)

25.309374465859957


In [6]:
recovered_vertices2=recover_vertices(mixing_weights, vertices1, endmembers)
print (E_RMS_kavan2010(vertices2, recovered_vertices2, scale))

2.90071396789943e-09


In [7]:
recovered_vertices2=recover_vertices(mixing_weights_clip, vertices1, endmembers)
print (E_RMS_kavan2010(vertices2, recovered_vertices2, scale))

initials={'x': col_major(mixing_weights_clip.reshape((-1,len(endmembers)))).reshape((-1,1))}
print (initials['x'].shape)

start=time.time()

res=run(mesh0, mesh1_list, weights, endmembers, mixing_weights.ravel(), grad_zero_indices, initials=initials)

res=res.reshape(mixing_weights.shape)
end=time.time()
print("using time: ", end-start)

recovered_vertices2=recover_vertices(res, vertices1, endmembers)
print (E_RMS_kavan2010(vertices2, recovered_vertices2, scale))

11053.189538523577
(106575, 1)
(5075, 24)
(5075, 18)
5075 6 21
W_spatial 0.0
     pcost       dcost       gap    pres   dres
 0: -6.7439e+05 -9.9455e+05  3e+06  4e+00  3e-14
 1: -6.7398e+05 -9.6379e+05  2e+06  3e+00  3e-14
 2: -6.5363e+05 -8.9195e+05  2e+06  2e+00  3e-14
 3: -6.1482e+05 -7.3799e+05  1e+06  1e+00  2e-14
 4: -5.6338e+05 -5.7968e+05  1e+06  1e+00  2e-14
 5: -5.3459e+05 -3.4072e+05  1e+06  1e+00  2e-14
 6: -4.6803e+05  8.2022e+04  1e+06  9e-01  2e-14
 7: -4.4614e+05  5.0460e+05  1e+06  9e-01  2e-14
 8: -3.7439e+05  1.3931e+06  2e+06  9e-01  2e-14
 9: -3.1624e+05  2.0354e+06  2e+06  8e-01  2e-14
10: -2.0422e+05  3.2452e+06  2e+06  8e-01  2e-14
11: -8.2764e+04  4.4966e+06  2e+06  8e-01  3e-14
12:  3.9970e+05  7.3843e+06  3e+06  7e-01  3e-14
13:  5.5246e+05  8.3237e+06  3e+06  7e-01  3e-14
14:  1.2675e+06  1.2016e+07  4e+06  7e-01  3e-14
15:  1.6209e+06  1.4000e+07  4e+06  7e-01  3e-14
16:  2.7594e+06  1.8347e+07  6e+06  6e-01  3e-14
17:  3.0113e+06  1.9276e+07  6e+06  6e-01 

In [8]:
print (res.reshape((-1, len(endmembers)))[:3]) ### should have same zeros as initial input.
print (res.ravel()[grad_zero_indices].sum()) ### should be zero
print (res.ravel()[grad_zero_indices].max()) ### should be zero
print (res.sum())

[[ 0.11850882 -0.07429014 -0.03383018  0.00202295 -0.00737332  0.02974563
   0.14073151 -0.11425928  0.02723813  0.06317792 -0.10447811  0.25543346
  -0.11236641  0.00010973 -0.00068714  0.04396792 -0.10861816  0.72447324
   0.00133004 -0.00319004  0.15235343]
 [ 0.08921593 -0.00032336  0.07870554  0.01423591  0.00036857  0.08360401
   0.08777066  0.08919588  0.08493473 -0.00019055  0.09506711  0.09060728
   0.0892062   0.0000012  -0.00001094  0.00014508  0.09794443  0.01056797
   0.00001145 -0.00000227  0.08894517]
 [ 0.08921506 -0.00032321  0.078712    0.01422783  0.00036849  0.08360549
   0.08777048  0.08919501  0.08493516 -0.00019102  0.09506785  0.09060547
   0.08920496  0.0000012  -0.00001092  0.00014501  0.09794299  0.01057476
   0.00001142 -0.00000226  0.08894424]]
3313.2507007580944
1.1159941713018335
5075.000000000025


In [9]:
print (((1-res.sum(axis=-1))**2).sum()/len(res))
print (res.min(), res.max())

9.563453912079597e-25
-0.11717243200493661 1.1159941713018335


In [10]:
recover_diff= (res.dot(endmembers)-mixing_weights.dot(endmembers))
recover_diff= (recover_diff**2).sum()
print (recover_diff)

392102735161.399


In [11]:
# #### read gt per vertex transformation 
# groundtruth_names=glob.glob("./models/"+exampleName+"-all/"+exampleName+"-*.DMAT")
# print(groundtruth_names)

# gt_all=None
# for i in range(len(groundtruth_names)):
#     gt=load_DMAT(groundtruth_names[i])
#     gt=gt.T
#     gt=gt.reshape((-1,4,3))
#     gt=np.transpose(gt, (0,2,1)).reshape((-1,12))
#     if gt_all is None:
#         gt_all=gt
#     else:
#         gt_all=np.hstack((gt_all, gt))
    
# print(gt_all.shape)


In [12]:
#### read gt per vertex mixing weights 
groundtruth_names=glob.glob("./models/"+exampleName+"-all/"+exampleName+".DMAT")
print(groundtruth_names)
gt=load_DMAT(groundtruth_names[0])
gt_mixing_weights=gt.T.copy()
print (gt_mixing_weights.shape)

print (((gt_mixing_weights.sum(axis=-1)-1)**2).sum())

['./models/wolf-all/wolf.DMAT']
(5075, 21)
5.575027928616619e-29


In [13]:
### load groundtruth endmember
groundtruth_names=glob.glob("./models/"+exampleName+"-all/"+poses_folder+"/"+exampleName+"-*.Tmat")
print(groundtruth_names)

gt_handles=None
for i in range(len(groundtruth_names)):
    gt=load_Tmat(groundtruth_names[i])
    gt=gt.reshape((-1,4,3))
    gt=np.transpose(gt, (0,2,1)).reshape((-1,12))
    if gt_handles is None:
        gt_handles=gt
    else:
        gt_handles=np.hstack((gt_handles, gt))
    
print(gt_handles.shape)
gt_handles=gt_handles.reshape(len(groundtruth_names), -1, 12)

gt_handles=np.transpose(gt_handles.reshape(-1, len(groundtruth_names), 12), (1,0,2)) ### is this true?

print(gt_handles.shape)

for i in range(len(gt_handles)):
    print (gt_handles[i][2].reshape((3,4)))

test=gt_handles[1][2].reshape((3,4))[:,:3]
print (test.T.dot(test))

['./models/wolf-all/poses-1/wolf-1.Tmat', './models/wolf-all/poses-1/wolf-2.Tmat', './models/wolf-all/poses-1/wolf-3.Tmat', './models/wolf-all/poses-1/wolf-4.Tmat', './models/wolf-all/poses-1/wolf-5.Tmat', './models/wolf-all/poses-1/wolf-6.Tmat']
(21, 72)
(6, 21, 12)
[[1. 0. 0. 0.]
 [0. 1. 0. 0.]
 [0. 0. 1. 0.]]
[[ 0.963816   0.259295   0.0618401  0.769371 ]
 [-0.215014   0.893333  -0.394619   1.05772  ]
 [-0.157567   0.367044   0.916761   2.30098  ]]
[[1. 0. 0. 0.]
 [0. 1. 0. 0.]
 [0. 0. 1. 0.]]
[[1. 0. 0. 0.]
 [0. 1. 0. 0.]
 [0. 0. 1. 0.]]
[[1. 0. 0. 0.]
 [0. 1. 0. 0.]
 [0. 0. 1. 0.]]
[[1. 0. 0. 0.]
 [0. 1. 0. 0.]
 [0. 0. 1. 0.]]
[[ 0.99999966 -0.00000045 -0.00000019]
 [-0.00000045  0.99999904  0.00000028]
 [-0.00000019  0.00000028  0.99999908]]


In [14]:
### plot handles corresponding to gt mixing weights, recovered mixing weights, and post-processed mixing weights.
###

Vertices=vertices2[:,poses_ind*3:poses_ind*3+3]

gt_handle_position=gt_mixing_weights.T.dot(Vertices)/gt_mixing_weights.T.sum(axis=-1)[...,np.newaxis]
recovered_handle_position=mixing_weights.T.dot(Vertices)/mixing_weights.T.sum(axis=-1)[...,np.newaxis]
cliped_handle_position=mixing_weights_clip.T.dot(Vertices)/mixing_weights_clip.T.sum(axis=-1)[...,np.newaxis]
postprocessed_handle_position=res.T.dot(Vertices)/res.T.sum(axis=-1)[...,np.newaxis]

%matplotlib 
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D

fig= plt.figure()
ax=fig.add_subplot(111, projection='3d')

ax.scatter(gt_handle_position[:,0],gt_handle_position[:,1], gt_handle_position[:,2], c='k', s=50)
ax.scatter(recovered_handle_position[:,0],recovered_handle_position[:,1], recovered_handle_position[:,2], c='r', s=50)
# ax.scatter(cliped_handle_position[:,0],cliped_handle_position[:,1], cliped_handle_position[:,2], c='g', s=50)
ax.scatter(postprocessed_handle_position[:,0],postprocessed_handle_position[:,1], postprocessed_handle_position[:,2], c='b', s=50)
ax.scatter(Vertices[:,0],Vertices[:,1], Vertices[:,2], color='y', s=1, edgecolors='none')

plt.show()

/usr/local/lib/python3.6/site-packages/ipykernel_launcher.py:8: RuntimeWarning: invalid value encountered in true_divide
  


Using matplotlib backend: MacOSX
